# Домашнее задание к лекции "Базовые понятия статистики"

### Домашнее задание к лекции "Базовые понятия статистики"

Будем осуществлять работу с непростым набором данных о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

#### Задание 1. Загрузка данных

Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

#### Задание 2. Первичное изучение данных

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

#### Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.


### Оглавление
  
1. [Задание 1](#task1)

2. [Задание 2](#task2)  

3. [Задание 3](#task3)  

###### 1. Задание 1 <a id='task1'></a>


In [1]:
# Загружаем необходимые библиотеки
import pandas as pd
import numpy as np
import requests
import re


In [2]:
# Загружаем файл с базой данных
df = pd.read_csv('horse_data.csv')
df

,2,1,530101,38.50,66,28,3,3.1,?,2.1,...,45.00,8.40,?.4,?.5,2.2,2.3,11300,00000,00000.1,2.4
0,1,1,534817,39.2,88,20,?,?,4,1,...,50,85,2,2,3,2,2208,0,0,2
1,2,1,530334,38.30,40,24,1,1,3,1,...,33.00,6.70,?,?,1,2,0,0,0,1
2,1,9,5290409,39.10,164,84,4,1,6,2,...,48.00,7.20,3,5.30,2,1,2208,0,0,1
3,2,1,530255,37.30,104,35,?,?,6,2,...,74.00,7.40,?,?,2,2,4300,0,0,2
4,2,1,528355,?,?,?,2,1,3,1,...,?,?,?,?,1,2,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1,1,533886,?,120,70,4,?,4,2,...,55,65,?,?,3,2,3205,0,0,2
295,2,1,527702,37.20,72,24,3,2,4,2,...,44.00,?,3,3.30,3,1,2208,0,0,1
296,1,1,529386,37.50,72,30,4,3,4,1,...,60.00,6.80,?,?,2,1,3205,0,0,2
297,1,1,530612,36.50,100,24,3,3,3,1,...,50.00,6.00,3,3.40,1,1,2208,0,0,1


В результате загрузки файла, теряется первая строка в виде заголовка столбцов.

Создадим заголовки для столбцов и вставим .

In [3]:
# Открываем файл с информацией о заголовках
f = open('horse_data.names')
pattern_t = r'\s+\d+[:]\s+'
title_ = []
for line in f:
    if re.findall(pattern_t, line):
        title_.append(re.sub(r'\s+\d+[:]\s+(\D*)\n', r'\1', line))
# Почти вручную корректируем заголовки столбцов))))
for i , line in enumerate(title_):
    if line == ' 25, 26,type of lesion':
        title_[i] = 'type of lesion'
        m = title_[i+1]
        title_[i+1] = 'type of lesion B'
        title_.append('type of lesion C')
        title_.append(m)
        break
# Вставляем названия столбцов
df.columns = title_
df

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
0,1,1,534817,39.2,88,20,?,?,4,1,...,50,85,2,2,3,2,2208,0,0,2
1,2,1,530334,38.30,40,24,1,1,3,1,...,33.00,6.70,?,?,1,2,0,0,0,1
2,1,9,5290409,39.10,164,84,4,1,6,2,...,48.00,7.20,3,5.30,2,1,2208,0,0,1
3,2,1,530255,37.30,104,35,?,?,6,2,...,74.00,7.40,?,?,2,2,4300,0,0,2
4,2,1,528355,?,?,?,2,1,3,1,...,?,?,?,?,1,2,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1,1,533886,?,120,70,4,?,4,2,...,55,65,?,?,3,2,3205,0,0,2
295,2,1,527702,37.20,72,24,3,2,4,2,...,44.00,?,3,3.30,3,1,2208,0,0,1
296,1,1,529386,37.50,72,30,4,3,4,1,...,60.00,6.80,?,?,2,1,3205,0,0,2
297,1,1,530612,36.50,100,24,3,3,3,1,...,50.00,6.00,3,3.40,1,1,2208,0,0,1


Первая строка потеряна.

Заголовки для столбцов вручную вставим строкой в файл с данными.

In [4]:
# подготовим строку для вставки 
for i,l in enumerate(title_):
    a = ','
    if i == 0:
        titl = title_[0]
    else:
        titl += a + title_[i]
titl

'surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,pain,peristalsis,abdominal distension,nasogastric tube,nasogastric reflux,nasogastric reflux PH,rectal examination - feces,abdomen,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data'

Файл с вставленными заголовками -----  horse_data_t.txt

In [5]:
# Загружаем в датафрейм файл с заголовками и заменяем пропуски на NaN
missing_values = ['?', 0, '00000', 'n/a', 'na']
df = pd.read_csv('horse_data_t.txt', na_values = missing_values)
df

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0,NaN,2.0,...,45.0,8.4,NaN,NaN,2.0,2,11300.0,NaN,NaN,2
1,1.0,1,534817,39.2,88.0,20.0,NaN,NaN,4.0,1.0,...,50.0,85.0,2.0,2.0,3.0,2,2208.0,NaN,NaN,2
2,2.0,1,530334,38.3,40.0,24.0,1.0,1.0,3.0,1.0,...,33.0,6.7,NaN,NaN,1.0,2,NaN,NaN,NaN,1
3,1.0,9,5290409,39.1,164.0,84.0,4.0,1.0,6.0,2.0,...,48.0,7.2,3.0,5.3,2.0,1,2208.0,NaN,NaN,1
4,2.0,1,530255,37.3,104.0,35.0,NaN,NaN,6.0,2.0,...,74.0,7.4,NaN,NaN,2.0,2,4300.0,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,1,533886,NaN,120.0,70.0,4.0,NaN,4.0,2.0,...,55.0,65.0,NaN,NaN,3.0,2,3205.0,NaN,NaN,2
296,2.0,1,527702,37.2,72.0,24.0,3.0,2.0,4.0,2.0,...,44.0,NaN,3.0,3.3,3.0,1,2208.0,NaN,NaN,1
297,1.0,1,529386,37.5,72.0,30.0,4.0,3.0,4.0,1.0,...,60.0,6.8,NaN,NaN,2.0,1,3205.0,NaN,NaN,2
298,1.0,1,530612,36.5,100.0,24.0,3.0,3.0,3.0,1.0,...,50.0,6.0,3.0,3.4,1.0,1,2208.0,NaN,NaN,1


Первая строка восстановлена

In [6]:
# Общая  информация по столбцам
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   surgery?                     299 non-null    float64
 1   Age                          300 non-null    int64  
 2   Hospital Number              300 non-null    int64  
 3   rectal temperature           240 non-null    float64
 4   pulse                        276 non-null    float64
 5   respiratory rate             242 non-null    float64
 6   temperature of extremities   244 non-null    float64
 7   peripheral pulse             231 non-null    float64
 8   mucous membranes             253 non-null    float64
 9   capillary refill time        268 non-null    float64
 10  pain                         245 non-null    float64
 11  peristalsis                  256 non-null    float64
 12  abdominal distension         244 non-null    float64
 13  nasogastric tube    

In [7]:
# Определим процент пропусков
(df.isna().mean() * 100).round(2)

surgery?                        0.33
Age                             0.00
Hospital Number                 0.00
rectal temperature             20.00
pulse                           8.00
respiratory rate               19.33
temperature of extremities     18.67
peripheral pulse               23.00
mucous membranes               15.67
capillary refill time          10.67
pain                           18.33
peristalsis                    14.67
abdominal distension           18.67
nasogastric tube               34.67
nasogastric reflux             35.33
nasogastric reflux PH          82.33
rectal examination - feces     34.00
abdomen                        39.33
packed cell volume              9.67
total protein                  11.00
abdominocentesis appearance    55.00
abdomcentesis total protein    66.00
outcome                         0.33
surgical lesion?                0.00
type of lesion                 18.67
type of lesion B               97.67
type of lesion C               99.33
c

In [8]:
# Pассчитаем базовые статистики
df.describe()

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
count,299.000000,300.000000,3.000000e+02,240.000000,276.000000,242.000000,244.000000,231.000000,253.000000,268.000000,...,271.000000,267.000000,135.000000,102.000000,299.000000,300.000000,244.000000,7.000000,2.00000,300.000000
mean,1.397993,1.640000,1.085889e+06,38.167917,71.913043,30.417355,2.348361,2.017316,2.853755,1.305970,...,46.295203,24.456929,2.037037,3.019608,1.551839,1.363333,4497.393443,3866.857143,1104.50000,1.670000
std,0.490305,2.173972,1.529801e+06,0.732289,28.630557,17.642231,1.045054,1.042428,1.620294,0.477629,...,10.419335,27.475009,0.804905,1.968567,0.737187,0.481763,5664.130616,1997.259909,1561.99888,0.470998
min,1.000000,1.000000,5.184760e+05,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000,1.000000,...,23.000000,3.300000,1.000000,0.100000,1.000000,1.000000,300.000000,1400.000000,0.00000,1.000000
25%,1.000000,1.000000,5.289040e+05,37.800000,48.000000,18.500000,1.000000,1.000000,1.000000,1.000000,...,38.000000,6.500000,1.000000,2.000000,1.000000,1.000000,2208.000000,3111.000000,552.25000,1.000000
50%,1.000000,1.000000,5.303055e+05,38.200000,64.000000,24.500000,3.000000,2.000000,3.000000,1.000000,...,45.000000,7.500000,2.000000,2.250000,1.000000,1.000000,3111.000000,3111.000000,1104.50000,2.000000
75%,2.000000,1.000000,5.347275e+05,38.500000,88.000000,36.000000,3.000000,3.000000,4.000000,2.000000,...,52.000000,57.000000,3.000000,3.900000,2.000000,2.000000,4205.000000,4612.000000,1656.75000,2.000000
max,2.000000,9.000000,5.305629e+06,40.800000,184.000000,96.000000,4.000000,4.000000,6.000000,3.000000,...,75.000000,89.000000,3.000000,10.100000,3.000000,2.000000,41110.000000,7111.000000,2209.00000,2.000000


Оставляем 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [9]:
sort_col = ['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']
df1 = df[sort_col]
df1

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


#### 2. Задание 2 <a id='task2'></a>

2. Первичное изучение данных

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [10]:
# Общая  информация по столбцам
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    float64
 1   Age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [11]:
# Определим процент пропусков
(df1.isna().mean() * 100).round(2)

surgery?                       0.33
Age                            0.00
rectal temperature            20.00
pulse                          8.00
respiratory rate              19.33
temperature of extremities    18.67
pain                          18.33
outcome                        0.33
dtype: float64

In [12]:
# Pассчитаем базовые статистики
df1.describe()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


По результатам максимальных значений, можно сказать что нереферентные значения присутствуют в столбце :  Age.


## Проанализируем столбцы


## surgery?

1: операция?

1 = Да, ему сделали операцию

2 = Его лечили без хирургического вмешательства

Тип данных качественный бинарный

Пропущено одно значение

Выбросов в этой колонке нет.

In [13]:
# Определим уникальные значения в столбце и их соотношение
df1['surgery?'].value_counts()

1.0    180
2.0    119
Name: surgery?, dtype: int64

In [14]:
# Определим базовые статистики
df1['surgery?'].describe()

count    299.000000
mean       1.397993
std        0.490305
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        2.000000
Name: surgery?, dtype: float64

In [15]:
# Определим процент пропусков
(df1['surgery?'].isna().mean() * 100).round(2)

0.33

## 'Age'

2: Возраст

1 = Взрослая лошадь 

2 = Молодые (< 6 месяцев)

Тип данных качественный бинарный (порядковый).

Пропущено одно значение

Выбросы  присутствуют в виде ошибки ввода данных.

In [16]:
# Определим уникальные значения в столбце и их соотношение
df1['Age'].value_counts()

1    276
9     24
Name: Age, dtype: int64

Возможные значения в столбце Age (Возраст) :

1 = Взрослая лошадь

2 = Молодые (< 6 месяцев)

Значение 9 невозможно. Поскольку 2 ни разу не встретилась, Можно сделать предположение, что вместо 2 проставлены 9

Заменим 9 на 2

In [17]:
df.loc[(df.Age == 9), 'Age'] = 2
# Перезапишем данные в df1
df1 = df[sort_col]

In [18]:
# Определим уникальные значения в столбце и их соотношение
df1['Age'].value_counts()

1    276
2     24
Name: Age, dtype: int64

In [19]:
# Pассчитаем базовые статистики
df1['Age'].describe()

count    300.000000
mean       1.080000
std        0.271746
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        2.000000
Name: Age, dtype: float64

In [20]:
# Определим процент пропусков
(df1['Age'].isna().mean() * 100).round(2)

0.0

Пропусков в этой колонке нет

## rectal temperature	

4: ректальная температура

- линейный

- в градусах Цельсия.

- Повышенная температура может возникнуть из-за инфекции.

- температура может быть снижена, когда животное находится в состоянии позднего шока

- нормальная температура составляет 37,8

- этот параметр обычно меняется по мере развития проблемы
например, может начаться нормально, затем стать повышенным из-
за поражения, возвращаясь в нормальный диапазон, когда
лошадь впадает в шок

Тип  данных количественные непрерывные


In [21]:
# Pассчитаем базовые статистики
df['rectal temperature'].describe()

count    240.000000
mean      38.167917
std        0.732289
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64

In [22]:
height_range = df1['rectal temperature'].max() - df1['rectal temperature'].min()
print('Pазмах – разница между минимальным и максимальным значением:', height_range)
print('Значение моды: ', df1['rectal temperature'].round().mode()[0])
# первый и третий квартили
Q1 = df1['rectal temperature'].quantile(0.25)
Q3 = df1['rectal temperature'].quantile(0.75)
# межквартильный размах
IQR = Q3 - Q1
print('межквартильный размах:', IQR)

Pазмах – разница между минимальным и максимальным значением: 5.399999999999999
Значение моды:  38.0
межквартильный размах: 0.7000000000000028


In [23]:
# Все моды
df1['rectal temperature'].value_counts().head(10)

38.0    26
38.5    19
38.3    18
37.8    17
38.2    16
37.5    12
38.1    12
38.6    12
38.4    11
37.6     7
Name: rectal temperature, dtype: int64

In [24]:
# Определим процент пропусков
print('процент пропусков:', (df1['rectal temperature'].isna().mean() * 100).round(2))

процент пропусков: 20.0


Hайдем выбросы через межквартильный размах

In [25]:
q1 = df1['rectal temperature'].quantile(0.25)
q3 = df1['rectal temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
print('нижний предел:', lower_bound)
upper_bound = q3 + (1.5 * iqr)
print('верхний предел', upper_bound)
remove_outliers = df1[df1['rectal temperature'].between(lower_bound, upper_bound, inclusive=True)].sort_values(['rectal temperature'])
remove_outliers

нижний предел: 36.74999999999999
верхний предел 39.550000000000004


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
113,1.0,1,36.8,60.0,28.0,NaN,NaN,2.0
277,2.0,1,36.9,50.0,40.0,2.0,1.0,1.0
70,2.0,1,37.0,60.0,20.0,3.0,3.0,3.0
292,1.0,1,37.0,66.0,20.0,1.0,4.0,2.0
190,1.0,1,37.1,40.0,8.0,NaN,3.0,1.0
...,...,...,...,...,...,...,...,...
165,1.0,1,39.4,120.0,48.0,NaN,NaN,3.0
231,2.0,1,39.5,92.0,28.0,3.0,5.0,2.0
287,2.0,2,39.5,84.0,30.0,NaN,NaN,1.0
117,2.0,1,39.5,NaN,NaN,3.0,3.0,3.0


In [26]:
# Выбросы
v_st = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
v_st

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
5,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
7,1.0,1,NaN,60.0,NaN,3.0,NaN,2.0
8,2.0,1,NaN,80.0,36.0,3.0,4.0,3.0
16,1.0,2,NaN,128.0,36.0,3.0,4.0,2.0
20,1.0,1,39.9,72.0,60.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...
274,1.0,1,NaN,76.0,NaN,NaN,NaN,3.0
281,2.0,1,40.0,78.0,NaN,3.0,2.0,2.0
293,1.0,1,NaN,78.0,24.0,3.0,NaN,3.0
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0


In [27]:
# Распределение выбросов
v_st['rectal temperature'].value_counts()

36.5    2
40.3    2
36.0    1
35.4    1
36.6    1
39.7    1
40.0    1
36.4    1
40.8    1
39.9    1
36.1    1
39.6    1
Name: rectal temperature, dtype: int64

Выбросы не значительные и лежат в интервале реальных значений. Их нужно оставить без изменений

## pulse

5: пульс

- линейный

- частота сердечных сокращений в ударах в минуту

- является отражением состояния сердца: 30 -40-это нормально для взрослых

- редко бывает, чтобы скорость был ниже нормы, хотя спортивные лошади
могут иметь пульс 20-25

- у животных с болезненными поражениями или страдающим от шока кровообращения
может быть повышенная частота сердечных сокращений

Тип  данных количественные непрерывные


In [28]:
# Pассчитаем базовые статистики
df1['pulse'].describe()

count    276.000000
mean      71.913043
std       28.630557
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

In [29]:
height_range = df1['pulse'].max() - df1['pulse'].min()
print('Pазмах – разница между минимальным и максимальным значением:', height_range)
print('Значение моды: ', df1['pulse'].round().mode()[0])
Q1 = df1['pulse'].quantile(0.25)
Q3 = df1['pulse'].quantile(0.75)
IQR = Q3 - Q1
print('межквартильный размах:', IQR)

Pазмах – разница между минимальным и максимальным значением: 154.0
Значение моды:  48.0
межквартильный размах: 40.0


In [30]:
# Все моды
df1['pulse'].value_counts().head(10)

48.0     28
60.0     25
40.0     18
88.0     12
52.0     12
44.0     11
72.0     11
100.0    11
120.0    10
42.0     10
Name: pulse, dtype: int64

In [31]:
# Определим процент пропусков
print('процент пропусков:', (df1['pulse'].isna().mean() * 100).round(2))

процент пропусков: 8.0


Hайдем выбросы через межквартильный размах

In [32]:
q1 = df1['pulse'].quantile(0.25)
q3 = df1['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
print('нижний предел:', lower_bound)
upper_bound = q3 + (1.5 * iqr)
print('верхний предел', upper_bound)
remove_outliers = df1[df1['pulse'].between(lower_bound, upper_bound, inclusive=True)].sort_values(['pulse'])
remove_outliers

нижний предел: -12.0
верхний предел 148.0


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
130,1.0,1,37.8,30.0,12.0,NaN,NaN,2.0
232,1.0,1,38.5,30.0,18.0,NaN,NaN,1.0
203,2.0,1,37.2,36.0,9.0,1.0,2.0,1.0
276,1.0,1,38.0,36.0,16.0,3.0,4.0,3.0
242,1.0,1,NaN,36.0,32.0,1.0,5.0,2.0
...,...,...,...,...,...,...,...,...
148,1.0,1,38.3,132.0,NaN,NaN,2.0,1.0
135,1.0,2,38.1,136.0,48.0,3.0,5.0,2.0
44,1.0,1,35.4,140.0,24.0,3.0,4.0,3.0
103,1.0,2,38.0,140.0,68.0,1.0,3.0,1.0


In [33]:
# Выбросы
v_st = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
v_st

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
5,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
41,2.0,2,39.0,150.0,72.0,NaN,NaN,1.0
52,2.0,1,NaN,NaN,NaN,1.0,1.0,1.0
55,1.0,2,38.6,160.0,20.0,3.0,3.0,2.0
58,1.0,1,NaN,NaN,20.0,4.0,5.0,2.0
74,1.0,2,NaN,NaN,NaN,NaN,NaN,2.0
93,2.0,1,NaN,NaN,NaN,3.0,5.0,2.0
115,2.0,1,NaN,NaN,40.0,3.0,3.0,1.0
117,2.0,1,39.5,NaN,NaN,3.0,3.0,3.0


In [34]:
# Распределение выбросов
v_st['pulse'].value_counts()

150.0    2
160.0    1
164.0    1
184.0    1
Name: pulse, dtype: int64

Выбросов всего пять, находятся в вернем пределе возможных значений для новорожденных . Их нужно оставить без изменений, поскольку они не сильно влияют на какие-то результаты, и при этом могут быть реальными значениями.

In [35]:
v_st[['Age','pulse']].dropna()

,Age,pulse
3,2,164.0
41,2,150.0
55,2,160.0
255,2,184.0
275,2,150.0


Все значения выбросов у жеребят в возрасте до  6 месяцев

## respiratory rate

6: частота дыхания

- линейный

- нормальная скорость составляет от 8 до 10

- полезность сомнительна из-за больших колебаний

Тип  данных количественные непрерывные


In [36]:
# Pассчитаем базовые статистики
df1['respiratory rate'].describe()

count    242.000000
mean      30.417355
std       17.642231
min        8.000000
25%       18.500000
50%       24.500000
75%       36.000000
max       96.000000
Name: respiratory rate, dtype: float64

In [37]:
height_range = df1['respiratory rate'].max() - df1['respiratory rate'].min()
print('Pазмах – разница между минимальным и максимальным значением:', height_range)
print('Значение моды: ', df1['respiratory rate'].round().mode()[0])
Q1 = df1['respiratory rate'].quantile(0.25)
Q3 = df1['respiratory rate'].quantile(0.75)
IQR = Q3 - Q1
print('межквартильный размах:', IQR)

Pазмах – разница между минимальным и максимальным значением: 88.0
Значение моды:  20.0
межквартильный размах: 17.5


In [38]:
# Все моды
df1['respiratory rate'].value_counts().head(10)

20.0    28
24.0    27
16.0    22
30.0    19
12.0    19
40.0    17
36.0    16
28.0    13
32.0    11
18.0     8
Name: respiratory rate, dtype: int64

In [39]:
# Определим процент пропусков
print('процент пропусков:', (df1['respiratory rate'].isna().mean() * 100).round(2))

процент пропусков: 19.33


Hайдем выбросы через межквартильный размах

In [40]:
q1 = df1['respiratory rate'].quantile(0.25)
q3 = df1['respiratory rate'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
print('нижний предел:', lower_bound)
upper_bound = q3 + (1.5 * iqr)
print('верхний предел', upper_bound)
remove_outliers = df1[df1['respiratory rate'].between(lower_bound, upper_bound, inclusive=True)].sort_values(['respiratory rate'])
remove_outliers

нижний предел: -7.75
верхний предел 62.25


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
190,1.0,1,37.1,40.0,8.0,NaN,3.0,1.0
203,2.0,1,37.2,36.0,9.0,1.0,2.0,1.0
140,1.0,1,38.1,44.0,9.0,3.0,2.0,1.0
89,2.0,1,38.3,42.0,10.0,1.0,1.0,1.0
162,2.0,1,39.5,60.0,10.0,3.0,3.0,1.0
...,...,...,...,...,...,...,...,...
220,1.0,1,39.2,88.0,58.0,4.0,5.0,3.0
20,1.0,1,39.9,72.0,60.0,1.0,5.0,1.0
23,1.0,2,38.3,130.0,60.0,NaN,2.0,1.0
185,1.0,1,38.5,120.0,60.0,4.0,NaN,1.0


In [41]:
# Выбросы
v_st = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
v_st

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
5,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
7,1.0,1,NaN,60.0,NaN,3.0,NaN,2.0
9,2.0,2,38.3,90.0,NaN,1.0,5.0,1.0
29,2.0,1,37.7,48.0,NaN,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...
271,1.0,1,37.7,80.0,NaN,3.0,5.0,1.0
274,1.0,1,NaN,76.0,NaN,NaN,NaN,3.0
281,2.0,1,40.0,78.0,NaN,3.0,2.0,2.0
284,2.0,1,38.5,54.0,NaN,1.0,3.0,1.0


In [42]:
# Распределение выбросов
v_st['respiratory rate'].value_counts()

80.0    3
68.0    3
70.0    2
84.0    2
96.0    2
72.0    1
88.0    1
66.0    1
Name: respiratory rate, dtype: int64

In [43]:
# Количество выбросов
v_st['respiratory rate'].value_counts().sum()

15

Выбросов 15, находятся в вернем пределе возможных значений . Их нужно оставить без изменений, поскольку они не сильно влияют на какие-то результаты, и при этом могут быть реальными значениями. Больше половины выбросов принадлежат молодым жеребцам. В раннем возрасте у них повышенное значение. Кроме того после марша или выгонки , количество дыханий может сильно повышаться.

In [44]:
# Сопоставление возрастной характеристики и частотой дыхания в выбросах
v_st[['Age','respiratory rate']].dropna()

,Age,respiratory rate
3,2,84.0
39,2,96.0
41,2,72.0
82,2,80.0
84,1,80.0
103,2,68.0
106,1,96.0
120,1,66.0
125,1,68.0
208,1,80.0


## temperature of extremities


7: температура конечностей

- субъективный признак периферического кровообращения


- возможные значения:

1 = Нормальный

2 = Теплый

3 = Круто ( Cool)

4 = Холодные (Cold)

- от cool до холодных конечностей указывают на возможный шок (cool to cold extremities indicate possible shock)

- горячие конечности должны коррелировать с повышенной температурой прямой кишки.

Тип  данных порядковый качественный.



In [45]:
# Pассчитаем базовые статистики
df1['temperature of extremities'].describe()

count    244.000000
mean       2.348361
std        1.045054
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max        4.000000
Name: temperature of extremities, dtype: float64

In [46]:
height_range = df1['temperature of extremities'].max() - df1['temperature of extremities'].min()
print('Pазмах – разница между минимальным и максимальным значением:', height_range)
print('Значение моды: ', df1['temperature of extremities'].round().mode()[0])
Q1 = df1['temperature of extremities'].quantile(0.25)
Q3 = df1['temperature of extremities'].quantile(0.75)
IQR = Q3 - Q1
print('межквартильный размах:', IQR)

Pазмах – разница между минимальным и максимальным значением: 3.0
Значение моды:  3.0
межквартильный размах: 2.0


In [47]:
# Все моды
df1['temperature of extremities'].value_counts()

3.0    109
1.0     78
2.0     30
4.0     27
Name: temperature of extremities, dtype: int64

In [48]:
# Определим процент пропусков
print('процент пропусков:', (df1['temperature of extremities'].isna().mean() * 100).round(2))

процент пропусков: 18.67


Поскольку все значения находятся в референтном интервале, говорить о выбросах не корректно.

## pain

11: боль 

- субъективное суждение об уровне боли лошади

- возможные значения:

1 = тревога, боли нет

2 = подавлен

3 = прерывистая легкая боль

4 = прерывистая сильная боль

5 = постоянная сильная боль

- не следует рассматривать как упорядоченную или дискретную переменную!

- В общем, чем больнее, тем больше вероятность того, что это потребует хирургия

- предварительное лечение боли может в некоторой степени маскировать уровень боли

Тип  данных порядковый качественный.


In [49]:
# Pассчитаем базовые статистики
df1['pain'].describe()

count    245.00000
mean       2.95102
std        1.30794
min        1.00000
25%        2.00000
50%        3.00000
75%        4.00000
max        5.00000
Name: pain, dtype: float64

In [50]:
height_range = df1['pain'].max() - df1['pain'].min()
print('Pазмах – разница между минимальным и максимальным значением:', height_range)
print('Значение моды: ', df1['pain'].round().mode()[0])
Q1 = df1['pain'].quantile(0.25)
Q3 = df1['pain'].quantile(0.75)
IQR = Q3 - Q1
print('межквартильный размах:', IQR)

Pазмах – разница между минимальным и максимальным значением: 4.0
Значение моды:  3.0
межквартильный размах: 2.0


In [51]:
# Все моды
df1['pain'].value_counts()

3.0    67
2.0    59
5.0    42
4.0    39
1.0    38
Name: pain, dtype: int64

In [52]:
# Определим процент пропусков
print('процент пропусков:', (df1['pain'].isna().mean() * 100).round(2))

процент пропусков: 18.33


Поскольку все значения находятся в референтном интервале, говорить о выбросах не корректно.

## outcome

23: результат

- что в конце концов случилось с лошадью?

- возможные значения:

1 = прожил

2 = умер

3 = подвергся эвтаназии

Тип  данных порядковый качественный.


In [53]:
# Pассчитаем базовые статистики
df1['outcome'].describe()

count    299.000000
mean       1.551839
std        0.737187
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        3.000000
Name: outcome, dtype: float64

In [54]:
height_range = df1['outcome'].max() - df1['outcome'].min()
print('Pазмах – разница между минимальным и максимальным значением:', height_range)
print('Значение моды: ', df1['outcome'].round().mode()[0])
Q1 = df1['outcome'].quantile(0.25)
Q3 = df1['outcome'].quantile(0.75)
IQR = Q3 - Q1
print('межквартильный размах:', IQR)

Pазмах – разница между минимальным и максимальным значением: 2.0
Значение моды:  1.0
межквартильный размах: 1.0


In [55]:
# Все моды
df1['outcome'].value_counts()

1.0    178
2.0     77
3.0     44
Name: outcome, dtype: int64

In [56]:
# Определим процент пропусков
print('процент пропусков:', (df1['outcome'].isna().mean() * 100).round(2))

процент пропусков: 0.33


Поскольку все значения находятся в референтном интервале, говорить о выбросах не корректно.

###### 3. Задание 3 <a id='task3'></a>

#### Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [57]:
# Определим процент пропусков
(df1.isna().mean() * 100).round(2)

surgery?                       0.33
Age                            0.00
rectal temperature            20.00
pulse                          8.00
respiratory rate              19.33
temperature of extremities    18.67
pain                          18.33
outcome                        0.33
dtype: float64

In [58]:
# Рассчитать количество пропусков для всех выбранных столбцов
df1.isna().sum()

surgery?                       1
Age                            0
rectal temperature            60
pulse                         24
respiratory rate              58
temperature of extremities    56
pain                          55
outcome                        1
dtype: int64

#### Столбец 'surgery?'  ( операция?)

Один пропуск

Один пропуск крайне незначительно влияет на общие показатели статистик.

In [59]:
# Определим строку с пропуском  и выведем всю информацию по ней
df[pd.isnull(df['surgery?'])]

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
132,NaN,1,534572,38.0,48.0,20.0,3.0,1.0,3.0,1.0,...,43.0,73.0,2.0,1.0,NaN,1,3111.0,NaN,NaN,2


В колонке 'surgical lesion?' указывается , что у этой лошади имелось хирургическое повреждение. Это значит , что ретроспективно, была  проблема или повреждение хирургическими,
( все случаи либо прооперированы, либо вскрыты ) , так что
это значение и тип поражения всегда известны

Также в колонке 'type of lesion' описан и классифицирован тип поражения

В колонке 'abdomen' (брюшная полость) также значение 5 указывает  на хирургическое повреждение





In [60]:
df['abdomen'][132]

5.0

Заменим значение NaN в этой строке на 1 (Да, ему сделали операцию)

In [61]:
df['surgery?'].fillna(1, inplace=True)
print(df['surgery?'].isna().sum())
print(df['surgery?'].unique())

0
[2. 1.]


#### Столбец 'Age'  (Возраст)

Пропуска отсутствуют

#### Столбец 'rectal temperature'  (ректальная температура)

60 пропусков

In [62]:
# Определим строки с пропуском  и выведем всю информацию по ним
str_rt = df[pd.isnull(df['rectal temperature'])]
str_rt.head()

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
5,2.0,1,528355,NaN,NaN,NaN,2.0,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,1.0,2,NaN,NaN,NaN,2
7,1.0,1,529607,NaN,60.0,NaN,3.0,NaN,NaN,1.0,...,44.0,8.3,NaN,NaN,2.0,1,2208.0,NaN,NaN,2
8,2.0,1,530051,NaN,80.0,36.0,3.0,4.0,3.0,1.0,...,38.0,6.2,NaN,NaN,3.0,1,3205.0,NaN,NaN,2
16,1.0,2,5301219,NaN,128.0,36.0,3.0,3.0,4.0,2.0,...,53.0,7.8,3.0,4.7,2.0,2,1400.0,NaN,NaN,1
28,1.0,1,5279442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2,4300.0,NaN,NaN,2


Колонка 'temperature of extremities' (температура конечностей) должна коррелироваться с 'rectal temperature' (ректальная температура)
- субъективный признак периферического кровообращения
- возможные значения:
1 = Нормальный
2 = Теплый
3 = Круто
4 = Холодные
- холодные до холодных конечностей указывают на возможный шок
- горячие конечности должны коррелировать с повышенной температурой прямой кишки.



In [63]:
df[['rectal temperature','temperature of extremities']].head()

,rectal temperature,temperature of extremities
0,38.5,3.0
1,39.2,NaN
2,38.3,1.0
3,39.1,4.0
4,37.3,NaN


В 46 стрроках имеется информация о температура конечностей

Определим среднюю ректальную температуру для каждого значения температуры конечностей.


In [64]:
print(df1.groupby('temperature of extremities')['rectal temperature'].median())
#fill_median_by_gender = pd.read_csv('titanic.csv')


temperature of extremities
1.0    38.15
2.0    38.20
3.0    38.10
4.0    38.35
Name: rectal temperature, dtype: float64


In [65]:
# Перед заменой  Pассчитаем базовые статистики
df['rectal temperature'].describe() 

count    240.000000
mean      38.167917
std        0.732289
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64

In [66]:
# Произведём замену отсутствующих значений 'rectal temperature' средними значениями по 'temperature of extremities'
df['rectal temperature'].fillna(df.groupby('temperature of extremities')['rectal temperature'].transform('median'), inplace=True)
df[['rectal temperature','temperature of extremities']].head()

,rectal temperature,temperature of extremities
0,38.5,3.0
1,39.2,NaN
2,38.3,1.0
3,39.1,4.0
4,37.3,NaN


In [67]:
#   Pассчитаем базовые статистики после замены
df['rectal temperature'].describe() 
# существенных изменений не претерпели

count    286.000000
mean      38.165559
std        0.671561
min       35.400000
25%       37.800000
50%       38.100000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64

In [68]:
# Оставшиеся 14 пропусков заполним медианным значением (меньше влияние на общие статистики)
# fill_median = pd.read_csv('titanic.csv')

df['rectal temperature'].fillna(df['rectal temperature'].median(), inplace=True)
df['rectal temperature'].fillna(df['rectal temperature'].median()).isna().sum()

0

In [69]:
#   Pассчитаем базовые статистики после замен
df['rectal temperature'].describe() 
# существенных изменений не претерпели

count    300.000000
mean      38.162500
std        0.655796
min       35.400000
25%       37.900000
50%       38.100000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64

### Столбец 'pulse'  (пульс)
  
24 пропускa

In [70]:
print(df['pulse'].isna().sum())
print(df['pulse'].unique())

24
[ 66.  88.  40. 164. 104.  nan  48.  60.  80.  90.  72.  42.  92.  76.
  96. 128.  64. 110. 130. 108. 100. 112.  52. 146. 150. 120. 140.  84.
  46. 114. 160.  54.  56.  38.  98.  50.  44.  70.  78.  49.  30. 136.
 132.  65.  86. 129.  68.  36.  45. 124. 184.  75.  82.]


In [71]:
# Определим строки с пропуском  и выведем всю информацию по ним
str_pl = df[pd.isnull(df['pulse'])]
str_pl.head()

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
5,2.0,1,528355,38.20,NaN,NaN,2.0,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,1.0,2,NaN,NaN,NaN,2
28,1.0,1,5279442,38.10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2,4300.0,NaN,NaN,2
52,2.0,1,529483,38.15,NaN,NaN,1.0,1.0,3.0,1.0,...,43.0,7.7,NaN,NaN,1.0,2,3111.0,NaN,NaN,2
56,1.0,1,528872,38.10,NaN,NaN,NaN,NaN,NaN,NaN,...,24.0,6.7,NaN,NaN,1.0,1,3112.0,NaN,NaN,2
58,1.0,1,528298,38.35,NaN,20.0,4.0,3.0,3.0,NaN,...,53.0,5.9,3.0,NaN,2.0,1,3205.0,NaN,NaN,1


Данные из колонки 'pulse'  коррелируются с 'peripheral pulse'

In [72]:
df[['pulse','peripheral pulse']]

,pulse,peripheral pulse
0,66.0,3.0
1,88.0,NaN
2,40.0,1.0
3,164.0,1.0
4,104.0,NaN
...,...,...
295,120.0,NaN
296,72.0,2.0
297,72.0,3.0
298,100.0,3.0


В 15 стрроках  с пропущенным 'pulse' имеется информация о 'peripheral pulse'

Определим средний 'peripheral pulse' для каждого значения 'pulse'.


In [73]:
print(df.groupby('peripheral pulse')['pulse'].median())

peripheral pulse
1.0    52.0
2.0    60.0
3.0    88.0
4.0    88.0
Name: pulse, dtype: float64


In [74]:
# Перед заменой  Pассчитаем базовые статистики
df['pulse'].describe() 

count    276.000000
mean      71.913043
std       28.630557
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

In [75]:
# Произведём замену отсутствующих значений 'pulse' медианными значениями в зависимости от 'peripheral pulse'
df['pulse'].fillna(df.groupby('peripheral pulse')['pulse'].transform('median'), inplace=True)
df[['pulse','peripheral pulse']].head()

,pulse,peripheral pulse
0,66.0,3.0
1,88.0,NaN
2,40.0,1.0
3,164.0,1.0
4,104.0,NaN


In [76]:
#   Pассчитаем базовые статистики после замен
df['pulse'].describe() 
# существенных изменений не претерпели

count    291.000000
mean      72.000000
std       28.169805
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

In [77]:
# Оставшиеся 9 пропусков заполним медианным значением (меньше влияние на общие статистики)
df['pulse'].fillna(df['pulse'].median(), inplace=True)
df['pulse'].fillna(df['pulse'].median()).isna().sum()

0

In [78]:
#   Pассчитаем базовые статистики после замен
df['pulse'].describe() 
# Изменений не претерпели

count    300.000000
mean      71.760000
std       27.776263
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

### Столбец 'respiratory rate'  ( частота дыхания)
 
- полезность сомнительна из-за больших колебаний

58 пропусков

In [79]:
print(df['respiratory rate'].isna().sum())
print(df['respiratory rate'].unique())

58
[28. 20. 24. 84. 35. nan 16. 36. 12. 52. 48. 21. 60. 34. 42. 30. 96. 72.
 15. 44. 32. 22. 18. 80. 40. 10. 51. 68. 66.  9. 14. 90.  8. 23. 58. 13.
 70. 26. 88. 25. 50.]


In [80]:
# Определим строки с пропуском  и выведем всю информацию по ним
str_rr = df[pd.isnull(df['respiratory rate'])]
str_rr.head()

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
5,2.0,1,528355,38.2,52.0,NaN,2.0,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,1.0,2,NaN,NaN,NaN,2
7,1.0,1,529607,38.1,60.0,NaN,3.0,NaN,NaN,1.0,...,44.0,8.3,NaN,NaN,2.0,1,2208.0,NaN,NaN,2
9,2.0,2,5299629,38.3,90.0,NaN,1.0,NaN,1.0,1.0,...,40.0,6.2,1.0,2.2,1.0,2,NaN,NaN,NaN,1
28,1.0,1,5279442,38.1,64.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2,4300.0,NaN,NaN,2
29,2.0,1,535415,37.7,48.0,NaN,2.0,1.0,1.0,1.0,...,45.0,76.0,NaN,NaN,1.0,2,NaN,NaN,NaN,2


В справочной литературе указывается на то что пульс и частота дыхания у лошадей в разных  возрастах взаимосвязаны. Примем что данные из колонки 'respiratory rate'  коррелируются с 'pulse' и 'Age'

In [81]:
df[['respiratory rate','pulse', 'Age']]

,respiratory rate,pulse,Age
0,28.0,66.0,1
1,20.0,88.0,1
2,24.0,40.0,1
3,84.0,164.0,2
4,35.0,104.0,1
...,...,...,...
295,70.0,120.0,1
296,24.0,72.0,1
297,30.0,72.0,1
298,24.0,100.0,1


In [82]:
df[['respiratory rate','pulse', 'Age']].isnull().sum()

respiratory rate    58
pulse                0
Age                  0
dtype: int64

Определим средний 'respiratory rate' для каждого значения 'pulse' и 'Age' .


In [83]:
print(df.groupby(['Age', 'pulse'])['respiratory rate'].median())

Age  pulse
1    30.0     15.0
     36.0     16.0
     38.0     20.0
     40.0     17.0
     42.0     25.0
              ... 
2    146.0    96.0
     150.0    61.0
     160.0    20.0
     164.0    84.0
     184.0    84.0
Name: respiratory rate, Length: 61, dtype: float64


In [84]:
# Перед заменой  Pассчитаем базовые статистики
df['respiratory rate'].describe() 

count    242.000000
mean      30.417355
std       17.642231
min        8.000000
25%       18.500000
50%       24.500000
75%       36.000000
max       96.000000
Name: respiratory rate, dtype: float64

In [85]:
# Произведём замену отсутствующих значений 'respiratory rate' его средними значениями в зависимости от 'pulse'и  'Age'
df['respiratory rate'].fillna(df.groupby(['Age', 'pulse'])['respiratory rate'].transform('median'), inplace=True)
df[['respiratory rate', 'pulse','Age']].head()

,respiratory rate,pulse,Age
0,28.0,66.0,1
1,20.0,88.0,1
2,24.0,40.0,1
3,84.0,164.0,2
4,35.0,104.0,1


In [86]:
#   Pассчитаем базовые статистики после замен
df['respiratory rate'].describe() 
# существенных изменений не претерпели , снизилось std.

count    294.000000
mean      30.239796
std       16.422675
min        8.000000
25%       20.000000
50%       28.000000
75%       36.000000
max       96.000000
Name: respiratory rate, dtype: float64

In [87]:
df[['respiratory rate','pulse', 'Age']].isnull().sum()

respiratory rate    6
pulse               0
Age                 0
dtype: int64

In [88]:
str_rr = df[pd.isnull(df['respiratory rate'])]
str_rr

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
9,2.0,2,5299629,38.3,90.0,NaN,1.0,NaN,1.0,1.0,...,40.0,6.2,1.0,2.2,1.0,2,NaN,NaN,NaN,1
50,2.0,1,529766,38.6,46.0,NaN,1.0,1.0,2.0,1.0,...,49.0,9.1,1.0,1.6,1.0,2,3111.0,NaN,NaN,1
74,1.0,2,5292929,38.1,64.0,NaN,NaN,NaN,NaN,NaN,...,37.0,4.9,NaN,NaN,2.0,1,11124.0,NaN,NaN,2
129,1.0,1,527698,38.4,49.0,NaN,NaN,NaN,1.0,1.0,...,44.0,7.6,NaN,NaN,1.0,1,2206.0,NaN,NaN,2
148,1.0,1,529272,38.3,132.0,NaN,NaN,3.0,6.0,2.0,...,57.0,8.0,NaN,5.2,1.0,1,2112.0,NaN,NaN,1
227,1.0,2,5289419,38.1,64.0,NaN,NaN,NaN,NaN,NaN,...,45.0,6.5,2.0,NaN,1.0,1,9000.0,NaN,NaN,1


В шести строках не нашлось соответствия комбинации 'pulse', 'Age' в имеющихся данных 'respiratory rate'

Заменим оставшиеся значения медианным значением (меньше влияние на общие статистики)

In [89]:
# Оставшиеся 9 пропусков заполним медианным значением (меньше влияние на общие статистики)
df['respiratory rate'].fillna(df['respiratory rate'].median(), inplace=True)
df['respiratory rate'].fillna(df['respiratory rate'].median()).isna().sum()

0

In [90]:
#   Pассчитаем базовые статистики после замен
df['respiratory rate'].describe() 
# существенных изменений не претерпели , ещё снизилось mean и std.

count    300.000000
mean      30.195000
std       16.260098
min        8.000000
25%       20.000000
50%       28.000000
75%       36.000000
max       96.000000
Name: respiratory rate, dtype: float64

 ### Столбец 'temperature of extremities' (  температура конечностей)

конечности должны коррелировать с повышенной температурой прямой кишки.

56 пропусков

In [91]:
print(df['temperature of extremities'].isna().sum())
print(df['temperature of extremities'].unique())

56
[ 3. nan  1.  4.  2.]


In [92]:
# Определим строки с пропуском  и выведем всю информацию по ним
str_te = df[pd.isnull(df['temperature of extremities'])]
str_te.head()

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
1,1.0,1,534817,39.2,88.0,20.0,NaN,NaN,4.0,1.0,...,50.0,85.0,2.0,2.0,3.0,2,2208.0,NaN,NaN,2
4,2.0,1,530255,37.3,104.0,35.0,NaN,NaN,6.0,2.0,...,74.0,7.4,NaN,NaN,2.0,2,4300.0,NaN,NaN,2
17,2.0,1,526639,37.5,48.0,24.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,2,NaN,NaN,NaN,2
23,1.0,2,534998,38.3,130.0,60.0,NaN,3.0,NaN,1.0,...,50.0,70.0,NaN,NaN,1.0,1,3111.0,NaN,NaN,2
25,2.0,1,529518,37.8,60.0,42.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,1.0,2,NaN,NaN,NaN,2


В описании сказано, что 'temperature of extremities' (температура конечностей) должна коррелировать с 'rectal temperature' (температурой прямой кишки).

In [93]:
df[['temperature of extremities','rectal temperature']]

,temperature of extremities,rectal temperature
0,3.0,38.50
1,NaN,39.20
2,1.0,38.30
3,4.0,39.10
4,NaN,37.30
...,...,...
295,4.0,38.35
296,3.0,37.20
297,4.0,37.50
298,3.0,36.50


In [94]:
df[['temperature of extremities','rectal temperature']].isnull().sum()

temperature of extremities    56
rectal temperature             0
dtype: int64

Определим среднюю 'rectal temperature'  для каждого значения 'temperature of extremities' .


In [95]:
print(df.groupby(['rectal temperature'])['temperature of extremities'].median().head())

rectal temperature
35.4    3.0
36.0    NaN
36.1    3.0
36.4    3.0
36.5    2.0
Name: temperature of extremities, dtype: float64


In [96]:
# Перед заменой  Pассчитаем базовые статистики
df['temperature of extremities'].describe() 

count    244.000000
mean       2.348361
std        1.045054
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max        4.000000
Name: temperature of extremities, dtype: float64

In [97]:
# Произведём замену отсутствующих значений 'respiratory rate' его средними значениями в зависимости от 'pulse'и  'Age'
df['temperature of extremities'].fillna(df.groupby(['rectal temperature'])['temperature of extremities'].transform('max'), inplace=True)
df[['temperature of extremities', 'rectal temperature']].head()

,temperature of extremities,rectal temperature
0,3.0,38.5
1,4.0,39.2
2,1.0,38.3
3,4.0,39.1
4,3.0,37.3


In [98]:
df['temperature of extremities'].value_counts()

3.0    142
1.0     78
4.0     48
2.0     30
Name: temperature of extremities, dtype: int64

In [99]:
#   Pассчитаем базовые статистики после замен
df['temperature of extremities'].describe() 
# существенных изменений не претерпели , снизилось std.

count    298.000000
mean       2.536913
std        1.047836
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max        4.000000
Name: temperature of extremities, dtype: float64

In [100]:
# В двух строках остались пропуски
df[['temperature of extremities', 'rectal temperature']].isnull().sum()

temperature of extremities    2
rectal temperature            0
dtype: int64

In [101]:
# Определим строки с пропуском  и выведем всю информацию по ним
str_te = df[pd.isnull(df['temperature of extremities'])]
str_te

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
113,1.0,1,527933,36.8,60.0,28.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,2.0,1,3205.0,NaN,NaN,1
141,2.0,1,522979,36.0,42.0,30.0,NaN,NaN,5.0,1.0,...,64.0,6.8,NaN,NaN,2.0,2,1400.0,NaN,NaN,2


Для таких низких значений 'rectal temperature' не нашлось соответствия с  'temperature of extremities'

Вставим  значение 4 = Холодные

In [102]:
# Оставшиеся 2 пропуска заполним  значением 4.0 (Холодные) 
df['temperature of extremities'].fillna(4.0, inplace=True)
df['temperature of extremities'].fillna(df['temperature of extremities'].median()).isna().sum()

0

In [103]:
#   Pассчитаем базовые статистики после всех замен
df['temperature of extremities'].unique() 
# изменений не претерпели 

array([3., 4., 1., 2.])


### Столбец 'pain' (  боль )

- субъективное суждение об уровне боли лошади

55 пропусков

In [104]:
print(df['pain'].isna().sum())
print(df['pain'].unique())

55
[ 5.  3.  2. nan  4.  1.]


In [105]:
# Определим строки с пропуском  и выведем всю информацию по ним
str_pa = df[pd.isnull(df['pain'])]
str_pa.head()

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
4,2.0,1,530255,37.3,104.0,35.0,3.0,NaN,6.0,2.0,...,74.0,7.4,NaN,NaN,2.0,2,4300.0,NaN,NaN,2
7,1.0,1,529607,38.1,60.0,28.0,3.0,NaN,NaN,1.0,...,44.0,8.3,NaN,NaN,2.0,1,2208.0,NaN,NaN,2
17,2.0,1,526639,37.5,48.0,24.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,2,NaN,NaN,NaN,2
19,2.0,1,532110,39.4,110.0,35.0,4.0,3.0,6.0,NaN,...,55.0,8.7,NaN,NaN,1.0,2,NaN,NaN,NaN,2
24,1.0,1,533692,38.1,60.0,12.0,3.0,3.0,3.0,1.0,...,51.0,65.0,NaN,NaN,1.0,1,3111.0,NaN,NaN,2


В описании сказано, что показатель 'pain' взаимосвязан с pulse и abdominal distension

In [106]:
df[['pain','pulse', 'abdominal distension']]

,pain,pulse,abdominal distension
0,5.0,66.0,4.0
1,3.0,88.0,2.0
2,3.0,40.0,1.0
3,2.0,164.0,4.0
4,NaN,104.0,NaN
...,...,...,...
295,2.0,120.0,NaN
296,4.0,72.0,3.0
297,4.0,72.0,3.0
298,3.0,100.0,3.0


In [107]:
df[['pain','pulse', 'abdominal distension']].isnull().sum()

pain                    55
pulse                    0
abdominal distension    56
dtype: int64

In [108]:
#len(df['respiratory rate'].unique())

In [109]:
#len(df['pulse'].unique())

Определим средний 'pain' для каждого значения 'pulse' и 'abdominal distension' .


In [110]:
print(df.groupby(['pulse', 'abdominal distension'])['pain'].median())

pulse  abdominal distension
36.0   1.0                     2.0
       2.0                     4.0
       3.0                     5.0
38.0   2.0                     3.0
40.0   1.0                     2.0
                              ... 
140.0  2.0                     3.0
150.0  2.0                     5.0
160.0  4.0                     3.0
164.0  4.0                     2.0
184.0  3.0                     4.0
Name: pain, Length: 102, dtype: float64


In [111]:
# Перед заменой  Pассчитаем базовые статистики
df['pain'].describe() 

count    245.00000
mean       2.95102
std        1.30794
min        1.00000
25%        2.00000
50%        3.00000
75%        4.00000
max        5.00000
Name: pain, dtype: float64

In [112]:
# Произведём замену отсутствующих значений 'respiratory rate' его средними значениями в зависимости от 'pulse'и 'abdominal distension'
df['pain'].fillna(df.groupby(['pulse', 'abdominal distension'])['pain'].transform('median'), inplace=True)
df[['pain', 'pulse', 'abdominal distension']].head()

,pain,pulse,abdominal distension
0,5.0,66.0,4.0
1,3.0,88.0,2.0
2,3.0,40.0,1.0
3,2.0,164.0,4.0
4,NaN,104.0,NaN


In [113]:
#   Pассчитаем базовые статистики после замен
df['pain'].describe() 
# существенных изменений не претерпели , снизилось std.

count    261.000000
mean       2.925287
std        1.305746
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: pain, dtype: float64

В 39 строках не нашлось соответствия комбинации 'pulse', 'abdominal distension' в имеющихся данных 'pain'



Определим средний 'respiratory rate' для каждого значения 'pulse'.


In [114]:
print(df.groupby(['pulse'])['pain'].median().head())

pulse
30.0    NaN
36.0    4.0
38.0    3.0
40.0    2.0
42.0    3.0
Name: pain, dtype: float64


In [115]:
# Перед заменой  Pассчитаем базовые статистики
df['pain'].describe() 

count    261.000000
mean       2.925287
std        1.305746
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: pain, dtype: float64

In [116]:
# Произведём замену отсутствующих значений 'respiratory rate' его средними значениями в зависимости от 'pulse'и 'abdominal distension'
df['pain'].fillna(df.groupby(['pulse'])['pain'].transform('median'), inplace=True)
df[['pain', 'pulse']].head()

,pain,pulse
0,5.0,66.0
1,3.0,88.0
2,3.0,40.0
3,2.0,164.0
4,3.0,104.0


In [117]:
#   Pассчитаем базовые статистики после замен
df['pain'].describe() 
# существенных изменений не претерпели , снизилось std.

count    295.000000
mean       2.923729
std        1.268447
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: pain, dtype: float64

В 5 строках не нашлось соответствия комбинации 'pulse' в имеющихся данных 'pain'

Заменим оставшиеся значения медианным значением (меньше влияние на общие статистики)

In [118]:
# Оставшиеся 9 пропусков заполним медианным значением (меньше влияние на общие статистики)
df['pain'].fillna(df['pain'].median(), inplace=True)
df['pain'].fillna(df['pain'].median()).isna().sum()

0

In [119]:
#   Pассчитаем базовые статистики после замен
df['pain'].describe() 
# существенных изменений не претерпели , ещё снизилось std.

count    300.000000
mean       2.925000
std        1.257835
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: pain, dtype: float64



### Столбец 'outcome' (   результат )

-  что в конце концов случилось с лошадью

1 пропуск

In [120]:
print(df['outcome'].isna().sum())
print(df['outcome'].unique())

1
[ 2.  3.  1. nan]


In [121]:
# Определим строку с пропуском  и выведем всю информацию по ней
str_ot = df[pd.isnull(df['outcome'])]
str_ot.head()

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
132,1.0,1,534572,38.0,48.0,20.0,3.0,1.0,3.0,1.0,...,43.0,73.0,2.0,1.0,NaN,1,3111.0,NaN,NaN,2


Одно пропущенное значение почти не скажется на статистиках, заменим пропуск 

In [122]:
df[['outcome','Age']]

,outcome,Age
0,2.0,1
1,3.0,1
2,1.0,1
3,2.0,2
4,2.0,1
...,...,...
295,3.0,1
296,3.0,1
297,2.0,1
298,1.0,1


Определим средний 'outcome' для каждого значения  'Age' .


In [123]:
print(df.groupby(['Age'])['outcome'].median())

Age
1    1.0
2    1.5
Name: outcome, dtype: float64


В строке с пропущенным значением -   'Age' = 1 , заменим пропуск на 1.0

In [124]:
# Оставшиеся 2 пропуска заполним  значением 1.0 (Холодные) 
df['outcome'].fillna('1.0', inplace=True)
df['outcome'].fillna(df['outcome'].median()).isna().sum()

0

In [125]:
#   Pассчитаем базовые статистики после всех замен
df['outcome'].describe() 
# изменений не претерпели 

count     300.0
unique      4.0
top         1.0
freq      178.0
Name: outcome, dtype: float64

Выводим требуемую таблицу с восемью столбцами без пропусков

In [126]:
df1 = df[sort_col]
df1

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.50,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.20,88.0,20.0,4.0,3.0,3.0
2,2.0,1,38.30,40.0,24.0,1.0,3.0,1.0
3,1.0,2,39.10,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.30,104.0,35.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,38.35,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.20,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.50,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.50,100.0,24.0,3.0,3.0,1.0


In [127]:
# Определим процент пропусков
(df1.isna().mean() * 100).round(2)

surgery?                      0.0
Age                           0.0
rectal temperature            0.0
pulse                         0.0
respiratory rate              0.0
temperature of extremities    0.0
pain                          0.0
outcome                       0.0
dtype: float64

In [128]:
df[['temperature of extremities','pulse']]

,temperature of extremities,pulse
0,3.0,66.0
1,4.0,88.0
2,1.0,40.0
3,4.0,164.0
4,3.0,104.0
...,...,...
295,4.0,120.0
296,3.0,72.0
297,4.0,72.0
298,3.0,100.0


In [129]:
df[['temperature of extremities','rectal temperature']].isnull().sum()

temperature of extremities    0
rectal temperature            0
dtype: int64

Определим среднюю 'rectal temperature'  для каждого значения 'temperature of extremities' .


In [130]:
print(df.groupby(['temperature of extremities'])['rectal temperature'].median())

temperature of extremities
1.0    38.15
2.0    38.20
3.0    38.10
4.0    38.30
Name: rectal temperature, dtype: float64


In [131]:
# Перед заменой  Pассчитаем базовые статистики
df['temperature of extremities'].describe() 

count    300.000000
mean       2.546667
std        1.051113
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max        4.000000
Name: temperature of extremities, dtype: float64

In [132]:
# Произведём замену отсутствующих значений 'respiratory rate' его средними значениями в зависимости от 'pulse'и  'Age'
#df['temperature of extremities'].fillna(df.groupby(['rectal temperature'])['temperature of extremities'].transform('median'), inplace=True)
#df[['temperature of extremities', 'rectal temperature']].head()

In [133]:
#   Pассчитаем базовые статистики после замен
df['temperature of extremities'].describe() 
# существенных изменений не претерпели , снизилось std.

count    300.000000
mean       2.546667
std        1.051113
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max        4.000000
Name: temperature of extremities, dtype: float64

In [134]:
# В двух строках остались пропуски
df[['temperature of extremities', 'rectal temperature']].isnull().sum()

temperature of extremities    0
rectal temperature            0
dtype: int64

In [135]:
# Определим строки с пропуском  и выведем всю информацию по ним
str_te = df[pd.isnull(df['temperature of extremities'])]
str_te

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data


Выводим требуемую таблицу с восемью столбцами без пропусков

In [136]:
df1 = df[sort_col]
df1

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.50,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.20,88.0,20.0,4.0,3.0,3.0
2,2.0,1,38.30,40.0,24.0,1.0,3.0,1.0
3,1.0,2,39.10,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.30,104.0,35.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,38.35,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.20,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.50,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.50,100.0,24.0,3.0,3.0,1.0


In [137]:
hjk = df.groupby(['Age', 'pulse'])['respiratory rate'].unique()
hjk[1][46]

array([28.])

In [138]:
#for i, vol in enumerate(hjk[1]):
   # vol.fillna(method='ffill', inplace=True)
   # print(i, vol)

In [139]:
#dfg = df.groupby(['Age', 'pulse'])['respiratory rate'].fillna(['respiratory rate'], method='ffill', inplace=True)

In [140]:
#len('objects around you are less')

In [141]:
#len(' significant then you think')

In [142]:
#dfg.head(30)

In [143]:
#df.fillna(method='ffill', inplace=True)
df

,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,type of lesion,type of lesion B,type of lesion C,cp_data
0,2.0,1,530101,38.50,66.0,28.0,3.0,3.0,NaN,2.0,...,45.0,8.4,NaN,NaN,2.0,2,11300.0,NaN,NaN,2
1,1.0,1,534817,39.20,88.0,20.0,4.0,NaN,4.0,1.0,...,50.0,85.0,2.0,2.0,3.0,2,2208.0,NaN,NaN,2
2,2.0,1,530334,38.30,40.0,24.0,1.0,1.0,3.0,1.0,...,33.0,6.7,NaN,NaN,1.0,2,NaN,NaN,NaN,1
3,1.0,2,5290409,39.10,164.0,84.0,4.0,1.0,6.0,2.0,...,48.0,7.2,3.0,5.3,2.0,1,2208.0,NaN,NaN,1
4,2.0,1,530255,37.30,104.0,35.0,3.0,NaN,6.0,2.0,...,74.0,7.4,NaN,NaN,2.0,2,4300.0,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,1,533886,38.35,120.0,70.0,4.0,NaN,4.0,2.0,...,55.0,65.0,NaN,NaN,3.0,2,3205.0,NaN,NaN,2
296,2.0,1,527702,37.20,72.0,24.0,3.0,2.0,4.0,2.0,...,44.0,NaN,3.0,3.3,3.0,1,2208.0,NaN,NaN,1
297,1.0,1,529386,37.50,72.0,30.0,4.0,3.0,4.0,1.0,...,60.0,6.8,NaN,NaN,2.0,1,3205.0,NaN,NaN,2
298,1.0,1,530612,36.50,100.0,24.0,3.0,3.0,3.0,1.0,...,50.0,6.0,3.0,3.4,1.0,1,2208.0,NaN,NaN,1


In [144]:
#type(sps)

In [145]:
#sps.describe() 

In [146]:
#DataFrame.fillna(self, value=None, method=None, axis=None, inplace=False, limit=None, downcast=None) 

In [147]:
#sps.unique().head(30)

In [148]:
#print(sps)

In [149]:
#fill_median = pd.read_csv('titanic.csv')

#fill_median['Age'].fillna(titanic['Age'].median(), inplace=True)
#fill_median['Age'].fillna(titanic['Age'].median()).isna().sum()

In [150]:
# Определим  оставшиеся строки с пропуском  и выведем всю информацию по ним
#print(df1.groupby('pulse')['rectal temperature'].median())

In [151]:
# titanic['Cabin'].fillna('no_info', inplace=True)
#print(df['rectal temperature'].isna().sum())
#print(df['rectal temperature'].unique())

In [152]:
# Оставшиеся строки с пропусками заменим 

In [153]:
#df['abdomen'][132]

Заменим значение NaN в этой строке на 1 (Да, ему сделали операцию)

In [154]:
#df1['surgery?'][132] = 1